In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'}}

In [2]:
model_name = "SVM_256_baseline_2class_B_neither"

In [3]:
import pandas as pd
import numpy as np
from joblib import dump, load

np.random.seed(seed=0)

In [4]:
train_data = pd.read_pickle(settings["train"]["pickle_path"])
valid_data = pd.read_pickle(settings["valid"]["pickle_path"])

train_data = pd.concat([train_data,valid_data],axis = 0)

test_data = pd.read_pickle(settings["test"]["pickle_path"])

In [5]:
columns = ['A_dist_bert', 'B_dist_bert',
       'A_pos_bert', 'B_pos_bert', 'pron_pos_bert', 'topic_A', 'topic_B', 
       'A_vector_bert_256_mean', 'B_vector_bert_256_mean',
       'pron_vector_bert_256_mean', 'product_vector_A_bert_256',
       'product_vector_B_bert_256']

In [6]:
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = np.array(train_data.label)


X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)
y_test = np.array(test_data.label)

In [8]:
from sklearn.model_selection import KFold 
from sklearn.svm import SVC
from sklearn.metrics import log_loss

score = []
pred_svm_tr = np.zeros((X_train.shape[0],2))

kf = KFold(n_splits=5) 
for train_index, test_index in kf.split(X_train):
    X_tr, X_ts = X_train[train_index], X_train[test_index]
    y_tr, y_ts = y_train[train_index], y_train[test_index]
    X_tr = X_tr[y_tr!=0]
    y_tr = y_tr[y_tr!=0]
    svm = SVC(C = 5.0,verbose=True,probability = True,gamma = "auto",class_weight='balanced').fit(X_tr, y_tr)
    pred_ts = svm.predict_proba(X_ts)
    s = log_loss(y_ts[y_ts!=0],pred_ts[y_ts!=0][:,[0,1]])
    score.append(s)
    pred_svm_tr[test_index] = pred_ts

    

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [9]:
score

[0.2773942460329124,
 0.3117223003090894,
 0.34147673203238904,
 0.32720050799184464,
 0.319874141686184]

In [10]:
np.array(score).mean()

0.31553358561048395

In [11]:
svm = SVC(C = 5.0,verbose=True,probability = True,gamma = "auto",class_weight='balanced').fit(X_train[y_train!=0], y_train[y_train!=0])

pred_svm = svm.predict_proba(X_test)

dump((pred_svm_tr,pred_svm), "../ensemble/"+ model_name +'+basic.joblib') 

[LibSVM]

['../ensemble/SVM_256_baseline_2class_B_neither+basic.joblib']

In [12]:
log_loss(y_test[y_test!=2],pred_svm[y_test!=2][:,[0,1]])

1.7811666817140217

In [13]:
X_train = np.concatenate([X_train,X_test],axis = 0)
y_train = np.concatenate([y_train,y_test],axis = 0)

svm = SVC(C = 5.0,verbose=True,probability = True,gamma = "auto",class_weight='balanced').fit(X_train[y_train!=0], y_train[y_train!=0])

[LibSVM]

In [14]:
dump(svm, model_name +'.joblib') 

['SVM_256_baseline_2class_B_neither.joblib']